In [1]:
Pkg.add("RDatasets")

INFO: Cloning cache of RDatasets from git://github.com/johnmyleswhite/RDatasets.jl.git
INFO: Installing RDatasets v0.1.2
INFO: Package database updated


In [7]:
using RDatasets
using StatsBase: logistic
data_set = "nodal"
nodal = dataset("boot", "nodal")
y = 2 * convert(Array{Float64},nodal[:R]) - 1
# X includes an intercept term
X = array(nodal[[:M, :Aged, :Stage, :Grade, :Xray, :Acid]])
X = convert(Array{Float64, 2}, X)

53x6 Array{Float64,2}:
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0
 ⋮                        ⋮  
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  1.0  1.0
 1.0  1.0  0.0  1.0  1.0  1.0
 1.0  1.0  0.0  0.0  1.0  1.0
 1.0  1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0
 1.0  0.0  1.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  1.0  0.0

In [3]:
y

53-element Array{Float64,1}:
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
  ⋮  
 -1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
 -1.0
  1.0
  1.0
 -1.0
 -1.0

In [6]:
n=1000
vcat(10:20:min(400,n), 420:40:min(600,n), 650:80:min(1000,n))


30-element Array{Int64,1}:
  10
  30
  50
  70
  90
 110
 130
 150
 170
 190
 210
 230
 250
   ⋮
 370
 390
 420
 460
 500
 540
 580
 650
 730
 810
 890
 970

In [13]:
using Distributions
srand(123)
function logit(z)
    1.0./(1.0.+exp(-z))
end

function fun(x::Array{Float64})
  x[1]^2
end
d=6;
C = eye(d);
Cinv = inv(C)
beta = reshape(rand(MvNormal(zeros(d),C)),(d,1))
x=X
# x= [randn(nobs,d-1) ones(nobs,1)]
# y=(rand(nobs).<reshape(logit(x *beta),nobs))*2-1;
xtp=x'

6x53 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0     0.0  1.0  1.0  1.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0     1.0  1.0  1.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0     0.0  1.0  0.0  1.0  0.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  1.0  1.0  0.0

In [16]:
function logit(z::Float64)
    1.0./(1.0.+exp(-z))
end
log_logit(z::Float64)= -log(1.0 .+ exp(-z))
grad_log_logit(z::Float64)=1.0-logit(z)

function log_posterior(beta_vec)
    log_prior = -0.5 * (beta_vec'*Cinv*beta_vec)[1]
    log_likelihood = sum([log_logit(z) for z in (reshape(x* beta_vec,nobs).*y)])
    return log_prior+log_likelihood
end

function gradmlogdensity(beta_vec::Array{Float64,2})
    grad=zeros(beta_vec)
    gradmloglike=zeros(beta_vec)

    grad+= Cinv * beta_vec;
    for i=1:nobs
        gradmloglike-=grad_log_logit(y[i]*(x[i,:]*beta_vec)[1])*y[i]*xtp[:,i];
    end
    return grad+gradmloglike;
end
function ddlogit(x::Float64)
    return exp(-x)/(1+exp(-x))^2
end

function hessmlogdensity(beta_vec::Array{Float64,2} )
    hess=zeros(d,d)
    hess+=Cinv;
    for i=1:nobs
        hess+=ddlogit(y[i]*(x[i,:]*beta_vec)[1])*y[i]^2*xtp[:,i]*x[i,:];
    end
    return hess;
end
function grad_log_posterior_sub(current_beta,chosen_indices)
    subobs= length(chosen_indices)
    log_prior_gradient= -Cinv* current_beta
    #gradient of likelihood = y*grad_log_logit(y*beta*x)*x
    #first step: compute y*grad_log_logit(y*beta*x)
    weights = y[chosen_indices].*map(grad_log_logit,(x[chosen_indices,:] * current_beta )[:].*y[chosen_indices])
    #second ztep: compute y*grad_log_logit(y*beta*x)*x

    return log_prior_gradient+(1.0*nobs)/subobs*(reshape(weights,1,subobs)* x[chosen_indices,:])'

end
function grad_log_posterior_sub_new(current_beta,chosen_indices)
    subobs= length(chosen_indices)
    log_prior_gradient= -Cinv* current_beta
    #gradient of likelihood = y*grad_log_logit(y*beta*x)*x
    #first step: compute y*grad_log_logit(y*beta*x)
    weights = y[chosen_indices].*map(grad_log_logit,( current_beta' *xtp[:,chosen_indices] )[:].*y[chosen_indices])
    #second ztep: compute y*grad_log_logit(y*beta*x)*x

    return log_prior_gradient+(1.0*nobs)/subobs*(reshape(weights,1,subobs)* x[chosen_indices,:])'

end
## S operator
function sgldstep(x,h,xi,cindex)
    x+h*grad_log_posterior_sub(x,cindex)+sqrt(2*h)*xi
end    
#     betamap=zeros(beta);

# step=hessmlogdensity(betamap)\gradmlogdensity(betamap)
# betamap=betamap-step;
# while norm(step)>1.0e-12
#     step=hessmlogdensity(betamap)\gradmlogdensity(betamap)
#     betamap=betamap-step;
#     println(norm(step))
#     println(betamap');
# end
# betamapv=vec(betamap)
#     #################### SGLD stuff 
# method can be one of :union :strat or :ind
# subset size 1-
# cost is 


sgldstep (generic function with 1 method)

In [25]:
m=1000
xs=zeros(d,m)
nobs=53
h0=0.04
subobs=20
for i=2:m
    
    cindF1=rand(1:nobs,subobs)

    xi1=randn(size(beta))
    xs[:,i]=sgldstep(xs[:,i-1],h0,xi1,cindF1)
end

In [26]:
xs

6x1000 Array{Float64,2}:
 0.0  -0.213414   -0.0224805  -0.053679  …  -1.99886   -1.66927   -1.86464 
 0.0  -0.567259   -0.407825   -0.728595     -0.959924  -0.340601  -0.462259
 0.0   0.26141     0.947841    1.28345       1.17146    1.70312    1.78824 
 0.0  -0.277121   -0.0567035   0.119224     -0.538465  -0.552004  -0.752566
 0.0   0.509268    0.527113    0.238367      2.68095    2.70335    2.73503 
 0.0  -0.0496297   0.604678    0.384957  …   0.226072   0.119866   0.590615